In [2]:
import os
import argparse
import logging
import os
import time
from tqdm import tqdm

import apex.amp as amp
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchvision.models import densenet121 as _densenet
import matplotlib.pyplot as plt
import os
import random

import apex.amp as amp
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, utils
from tqdm import tqdm

import cv2
import random
from sklearn.feature_extraction import image
from PIL import Image

cuda:1
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(torch.cuda.device_count())
#torch.cuda.set_device(1)

1


In [75]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

lr_max = 0.004
momentum = 0.9
num_classes = 5
model_name = "densenet"
feature_extract = False
root_dir = '/home/mtech/2020/satyaki_ghosh/datasets/VISION'
IMG_SIZE = 224
channels = 3
patches_per_photo = 2


vision_mean = (0.4333, 0.4635, 0.4767)
vision_std = (0.2687, 0.2536, 0.2593)
mu = torch.tensor(vision_mean).view(3,1,1).cuda()
std = torch.tensor(vision_std).view(3,1,1).cuda()
epsilon = (8 / 255.) / std
pgd_alpha = (2 / 255.) / std
upper_limit = ((1 - mu)/ std)
lower_limit = ((0 - mu)/ std) 
criterion = nn.CrossEntropyLoss()

In [59]:
def clamp(X, lower_limit, upper_limit):
    return torch.max(torch.min(X, upper_limit), lower_limit)

def attack_pgd(model, X, y, epsilon, alpha, attack_iters, restarts, opt=None):
    max_loss = torch.zeros_like(y).cuda()
    max_delta = torch.zeros_like(X).cuda()
    for zz in range(restarts):
        delta = torch.zeros_like(X).cuda()
        for i in range(len(epsilon)):
            delta[:, i, :, :].uniform_(-epsilon[i][0][0].item(), epsilon[i][0][0].item())
        delta.data = clamp(delta, lower_limit - X, upper_limit - X)
        delta.requires_grad = True
        for _ in range(attack_iters):
            output = model(X + delta)
            index = torch.where(output.max(1)[1] == y)
            if len(index[0]) == 0:
                break
            loss = criterion(output, y)
            if opt is not None:
                with amp.scale_loss(loss, opt) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            grad = delta.grad.detach()
            d = delta[index[0], :, :, :]
            g = grad[index[0], :, :, :]
            d = clamp(d + alpha * torch.sign(g), -epsilon, epsilon)
            d = clamp(d, lower_limit - X[index[0], :, :, :], upper_limit - X[index[0], :, :, :])
            delta.data[index[0], :, :, :] = d
            delta.grad.zero_()

        all_loss = F.cross_entropy(model(X+delta), y, reduction='none').detach()
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss.float(), all_loss)
    return max_delta

def evaluate_pgd(X, y, model, attack_iters, restarts):
    epsilon = (8 / 255.) / std   
    alpha = (2 / 255.) / std
    pgd_loss = 0
    pgd_acc = 0
    n = 0
    count = 0
    model.eval()
    X, y = X.cuda(), y.cuda()
    pgd_delta = attack_pgd(model, X, y, epsilon, alpha, attack_iters, restarts)
    with torch.no_grad():
        output = model(X + pgd_delta)
        loss = F.cross_entropy(output, y)
        pgd_loss += loss.item() * y.size(0)
        pgd_acc += (output.max(1)[1] == y).sum().item()
        n += y.size(0)
    return pgd_loss/n, pgd_acc/n, output

def evaluate_standard(X, y, model):
    test_loss = 0
    test_acc = 0
    n = 0
    model.eval()
    with torch.no_grad():
        X, y = X.cuda(), y.cuda()
        output = model(X)
        loss = F.cross_entropy(output, y)
        test_loss += loss.item() * y.size(0)
        test_acc += (output.max(1)[1] == y).sum().item()
        n += y.size(0)
    return test_loss/n, test_acc/n, output


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
    
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    """ Densenet
    """
    model_ft = models.densenet121(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    input_size = 224

    return model_ft, input_size

In [100]:
# load pretrained model

model, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
model = model.cuda()

PATH = '/home/mtech/2020/satyaki_ghosh/codes/fastAdvTraining/fast_adversarial/VISION/train_free_output/model_77.pth'

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
# opt.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.float()
model.eval()

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

6958981


In [94]:
import warnings
warnings.filterwarnings('ignore')
patches_per_photo = 100
images_per_model = 50

CATEGORIES = [
    "D01_Samsung_GalaxyS3Mini",
    "D02_Apple_iPhone4s",
    "D03_Huawei_P9",
    "D04_LG_D290",
    "D05_Apple_iPhone5c"]

total = 0
correct_images_normal = 0
correct_patches_normal = 0
correct_images_adv = 0
correct_patches_adv = 0


for camera_model in CATEGORIES:
    y = CATEGORIES.index(camera_model)
    path = os.path.join(os.path.join(root_dir, camera_model), 'images/nat')
    
    # shuffling the images to avoid similar/close images
    photos = os.listdir(path)
    random.shuffle(photos)
    test_images = photos[0:images_per_model]
    
    loop = tqdm(test_images, total = len(test_images))
    for image_name in loop:
        total += 1
        image_path = os.path.join(path, image_name)
        img = cv2.imread(image_path)
#         plt.imshow(img)
#         plt.show()

        pgd_preds = []
        std_preds = []

        patches = image.extract_patches_2d(img, (IMG_SIZE, IMG_SIZE), max_patches = patches_per_photo)
        y = torch.tensor(y, dtype=torch.long).reshape(1).cuda()

        for i in range(patches_per_photo) :                    
            X = patches[i]            
#             im = Image.fromarray(X)
#             plt.imshow(im)
#             plt.show()
            X = X.astype('float')
            X = torch.from_numpy(X)
            X = X.permute(2, 0, 1)
            X = X.reshape(-1,3,224,224).float()
            test_loss, test_acc, std_output = evaluate_standard(X, y, model)
            std_preds.append(std_output.max(1)[1])
            # pgd_loss, pgd_acc, pgd_output = evaluate_pgd(X, y, model, 50, 10)
            pgd_loss, pgd_acc, pgd_output = test_loss, test_acc, std_output
            pgd_preds.append(pgd_output.max(1)[1])
            if y == std_output.max(1)[1]:
                correct_patches_normal = correct_patches_normal + 1
            if y[0] == pgd_output.max(1)[1][0]:
                correct_patches_adv = correct_patches_adv + 1
            # print(f'Original label = {y[0]}, Standard Predicted label = {std_output.max(1)[1][0]}, PGD Predicted label = {pgd_output.max(1)[1][0]}') 
        
        normal_output = max(set(std_preds), key=std_preds.count)[0]
        adv_output = max(set(pgd_preds), key=pgd_preds.count)[0]
        
        if normal_output == y[0]:
            correct_images_normal = correct_images_normal + 1
        if adv_output == y[0]:
            correct_images_adv = correct_images_adv + 1            
    
        loop.set_postfix(img_acc = correct_images_normal / total,
                         patch_acc = correct_patches_normal / (total * patches_per_photo),
                         adv_img_acc = correct_images_adv / total,
                         adv_patch_acc = correct_patches_adv / (total * patches_per_photo)
                        )

#         print(f'Overall output of normal image (by simple voting) : {normal_output}')
#         print(f'Overall output of adversarial image (by simple voting) : {adv_output}')

print(f'Image-level normal accuracy: {correct_images_normal / total}')
print(f'Patch-level normal accuracy: {correct_patches_normal / (total * patches_per_photo)}')
print()
print(f'Image-level robust accuracy: {correct_images_adv / total}')
print(f'Patch-level robust accuracy: {correct_patches_adv / (total * patches_per_photo)}')

100%|███████████████████████████████████████████████████████████████████| 50/50 [10:15<00:00, 12.32s/it, adv_img_acc=0.204, adv_patch_acc=0.204, img_acc=0.204, patch_acc=0.204]

Image-level normal accuracy: 0.204
Patch-level normal accuracy: 0.20388

Image-level robust accuracy: 0.204
Patch-level robust accuracy: 0.20388


In [72]:
zz = torch.zeros_like(X, dtype=float)

In [73]:
zz[:,:,:].uniform_(-0.1,0.1)

tensor([[[[ 0.0940,  0.0841, -0.0643,  ...,  0.0049,  0.0987, -0.0890],
          [ 0.0105, -0.0587, -0.0486,  ..., -0.0997, -0.0105, -0.0616],
          [-0.0018, -0.0839,  0.0974,  ...,  0.0002, -0.0747, -0.0394],
          ...,
          [-0.0060,  0.0578,  0.0990,  ..., -0.0624, -0.0206,  0.0895],
          [ 0.0358, -0.0337,  0.0994,  ..., -0.0994,  0.0875,  0.0292],
          [-0.0589,  0.0325,  0.0957,  ...,  0.0313, -0.0214, -0.0574]],

         [[ 0.0416,  0.0290, -0.0298,  ...,  0.0935,  0.0965, -0.0036],
          [-0.0107,  0.0099,  0.0026,  ..., -0.0475,  0.0270, -0.0738],
          [ 0.0766, -0.0156, -0.0716,  ...,  0.0185, -0.0824,  0.0350],
          ...,
          [ 0.0516,  0.0168, -0.0805,  ..., -0.0767, -0.0673, -0.0529],
          [ 0.0164, -0.0160, -0.0567,  ...,  0.0893,  0.0451,  0.0888],
          [ 0.0271,  0.0580, -0.0780,  ...,  0.0712, -0.0442,  0.0213]],

         [[-0.0081,  0.0582,  0.0163,  ...,  0.0631,  0.0589, -0.0511],
          [-0.0422,  0.0237, -

In [74]:
zz.reshape(-1,3,224,224).shape

torch.Size([1, 3, 224, 224])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def patch_and_image_level_accuracy(model):
    patches_per_photo = 50
    images_per_model = 5

    CATEGORIES = [
        "D01_Samsung_GalaxyS3Mini",
        "D02_Apple_iPhone4s",
        "D03_Huawei_P9",
        "D04_LG_D290",
        "D05_Apple_iPhone5c"]

    total = 0
    correct_images_normal = 0
    correct_patches_normal = 0
    correct_images_adv = 0
    correct_patches_adv = 0


    for camera_model in CATEGORIES:
        y = CATEGORIES.index(camera_model)
        path = os.path.join(os.path.join(root_dir, camera_model), 'images/nat')

        # shuffling the images to avoid similar/close images
        photos = os.listdir(path)
        random.shuffle(photos)
        test_images = photos[0:images_per_model]

        loop = tqdm(test_images, total = len(test_images))
        for image_name in loop:
            total += 1
            image_path = os.path.join(path, image_name)
            img = cv2.imread(image_path)

            pgd_preds = []
            std_preds = []

            patches = image.extract_patches_2d(img, (IMG_SIZE, IMG_SIZE), max_patches = patches_per_photo)
            y = torch.tensor(y, dtype=torch.long).reshape(1).cuda()

            for i in range(patches_per_photo) :                    
                X = patches[i]            
                X = X.astype('float')
                X = torch.from_numpy(X)
                X = X.permute(2, 0, 1)
                X = X.reshape(-1,3,224,224).float()
                test_loss, test_acc, std_output = evaluate_standard(X, y, model)
                std_preds.append(std_output.max(1)[1])
                pgd_loss, pgd_acc, pgd_output = evaluate_pgd(X, y, model, 50, 10)
                pgd_preds.append(pgd_output.max(1)[1])
                if y == std_output.max(1)[1]:
                    correct_patches_normal = correct_patches_normal + 1
                if y[0] == pgd_output.max(1)[1][0]:
                    correct_patches_adv = correct_patches_adv + 1
                    
            normal_output = max(set(std_preds), key=std_preds.count)[0]
            adv_output = max(set(pgd_preds), key=pgd_preds.count)[0]

            if normal_output == y[0]:
                correct_images_normal = correct_images_normal + 1
            if adv_output == y[0]:
                correct_images_adv = correct_images_adv + 1            

            loop.set_postfix(img_acc = correct_images_normal / total,
                             patch_acc = correct_patches_normal / (total * patches_per_photo),
                             adv_img_acc = correct_images_adv / total,
                             adv_patch_acc = correct_patches_adv / (total * patches_per_photo)
                            )
            
    print(f'Image-level normal accuracy: {correct_images_normal / total}')
    print(f'Patch-level normal accuracy: {correct_patches_normal / (total * patches_per_photo)}')
    print()
    print(f'Image-level robust accuracy: {correct_images_adv / total}')
    print(f'Patch-level robust accuracy: {correct_patches_adv / (total * patches_per_photo)}')